# System Information

In [1]:
import sys

In [2]:
sys.version

'3.7.4 (default, Aug 13 2019, 20:35:49) \n[GCC 7.3.0]'

# Libraries

In [3]:
import pandas as pd
from collections import defaultdict

import string
import numpy as np

In [4]:
import dask.bag as db
from dask.distributed import Client

# Setup

Setup the dask cluster and some utility functions to help with map reduce:

In [5]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36799 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.45 GB


In [6]:
def f(row):
    d = {}
    col1, col2, col3 = row.col1, row.col2, row.col3
    
    d[(col1, col2)] = col3
    
    return d

In [7]:
def g(dict1, dict2):
    for k, v in dict2.items():
        dict1[k] += v
        
    return dict1

# Timings

Create the dataset:

In [8]:
n = 6000000

In [9]:
big_df = pd.DataFrame({
    "col1": np.random.choice(list(string.ascii_uppercase), n, replace=True),
    "col2": np.random.choice(list(string.ascii_lowercase), n, replace=True),
    "col3": np.random.randint(1, n, n)
})

Pandas solution with iterrows:

In [10]:
%%time

results1 = defaultdict(lambda : defaultdict(int))

for i, (col1, col2, col3) in big_df.iterrows():
    results1[col1][col2] += col3

CPU times: user 14min 23s, sys: 17.1 s, total: 14min 41s
Wall time: 14min 21s


Dask solution:

In [11]:
%%time

x = list(big_df.itertuples())
x = db.from_sequence(x)

results2 = x \
    .map(f) \
    .fold(g, initial=defaultdict(int)) \
    .compute()

CPU times: user 4min 2s, sys: 3.27 s, total: 4min 5s
Wall time: 4min 54s


Pandas solution with itertuples:

In [12]:
%%time

results3 = defaultdict(lambda : defaultdict(int))

for (_, col1, col2, col3) in big_df.itertuples():
    results3[col1][col2] += col3

CPU times: user 8.74 s, sys: 161 ms, total: 8.9 s
Wall time: 8.71 s


Pandas solution with custom data structure:

In [13]:
%%time

my_rows = zip(big_df["col1"], big_df["col2"], big_df["col3"])

results4 = defaultdict(lambda : defaultdict(int))

for i, (col1, col2, col3) in enumerate(my_rows):
    results4[col1][col2] += col3

CPU times: user 4.73 s, sys: 65.5 ms, total: 4.8 s
Wall time: 4.68 s


The itertuples solution with name set to None:

In [14]:
%%time

results5 = defaultdict(lambda : defaultdict(int))

for (_, col1, col2, col3) in big_df.itertuples(name=None):
    results5[col1][col2] += col3

CPU times: user 4.97 s, sys: 114 ms, total: 5.08 s
Wall time: 4.96 s


The most clever solution:

In [15]:
%%time

big_df.groupby(["col1", "col2"]).sum()

CPU times: user 657 ms, sys: 130 ms, total: 787 ms
Wall time: 755 ms


col3
col1 col2             
A    a     26784474391
     b     26772075810
     c     26469563880
     d     26261441982
     e     26441730736
...                ...
Z    v     26510916447
     w     26217424550
     x     27145925193
     y     26853841579
     z     26614820037

[676 rows x 1 columns]

In [16]:
results5["A"]["a"]

26784474391

Check all approaches gives identical results:

In [17]:
for (key1, key2), value in results2.items():
    assert results2[(key1, key2)] == results1[key1][key2]

assert results1 == results3
assert results3 == results4
assert results4 == results5